**************************************************INITIALIZATION*********************************************************

In [87]:
from tjmonopix.tjmonopix import TJMonoPix
import time
import tjmonopix.scans.simple_scan

chip = TJMonoPix()
chip['CONF']['DEF_CONF_N'] = 0
chip['CONF']['AB_SELECT'] = 1
chip['CONF'].write()
chip.init()

chip['data_rx'].CONF_START_FREEZE = 15 #default 3
chip['data_rx'].CONF_STOP_FREEZE = 100 #default 40
chip['data_rx'].CONF_START_READ = 35 #default 6
chip['data_rx'].CONF_STOP_READ = 37 #default 7
chip['data_rx'].CONF_STOP = 105 #default 45

chip.power_on()

chip['CONF']['RESET_BCID'] = 1
chip['CONF']['RESET'] = 1
chip['CONF'].write()

chip['CONF']['EN_BX_CLK'] = 1
chip['CONF']['EN_OUT_CLK'] = 1
chip['CONF'].write()
 
chip['CONF']['RESET_BCID'] = 0
chip['CONF']['RESET'] = 0
chip['CONF'].write()

chip.default_conf()

#-------------------------------------------------#
chip.set_icasn_dacunits(0,0)
chip.set_vreset_dacunits(35,0)
chip.set_ireset_dacunits(5,1,0)
chip.set_ithr_dacunits(30,0)
chip.set_idb_dacunits(50,0)

chip['CONF_SR']['EN_HV'].setall(False)
chip['CONF_SR']['EN_COMP'].setall(False)
chip['CONF_SR']['EN_PMOS'].setall(False)
chip['CONF_SR']['EN_PMOS_NOSF'].setall(False)
chip['CONF_SR']['EN_TEST_PATTERN'].setall(False)

chip['CONF_SR']['MASKD'].setall(False)
chip['CONF_SR']['MASKH'].setall(False)
chip['CONF_SR']['MASKV'].setall(False)

chip.write_conf()

chip['CONF']['DEF_CONF_N'] = 1
chip['CONF'].write()
#chip['BiasSF'].set_current(100, unit='uA') # SET THE SOURCE FOLLOWER READOUT CURRENT
chip.get_power_status()

2018-03-09 17:11:15,806 - [root           ] - ERROR   'NoneType' object has no attribute 'WriteExternal'
2018-03-09 17:11:15,818 - [root           ] - INFO    Found USB board(s): USBpix with ID 129 (FW 08)
2018-03-09 17:11:15,821 - [root           ] - INFO    Programming FPGA: tjmonopix_mioB.bit...
2018-03-09 17:11:23,987 - [root           ] - INFO    Success!
2018-03-09 17:11:24,434 - [root           ] - INFO    Found adapter card: General Purpose Analog Card with ID 12
2018-03-09 17:11:24,457 - [root           ] - INFO    Initializing fifo (firmware version: 2), module basil.HL.sram_fifo, base_addr 0x18000
2018-03-09 17:11:24,459 - [root           ] - INFO    Initializing gpio (firmware version: 0), module basil.HL.gpio, base_addr 0x10000
2018-03-09 17:11:24,462 - [root           ] - INFO    Initializing spi (firmware version: 2), module basil.HL.spi, base_addr 0x11000
2018-03-09 17:11:24,466 - [root           ] - INFO    Initializing inj (firmware version: 3), module basil.HL.pulse_

4099


{'VDDA [V]': 1.7825418966900703,
 'VDDA [mA]': 57.689059348560995,
 'VDDA_DAC [V]': 1.7981297475149105,
 'VDDA_DAC [mA]': 3.513856326790666,
 'VDDD [V]': 1.799880103792415,
 'VDDD [mA]': 0.501475573372531,
 'VDDP [V]': 1.788289688,
 'VDDP [mA]': 155.33580048907064}

****************************************************POWER_STATUS*********************************************************

In [88]:
chip['CONF_SR']['MASKD'].setall(False)
chip['CONF_SR']['MASKH'].setall(False)
chip['CONF_SR']['MASKV'].setall(False)
chip['CONF_SR']['EN_PMOS'].setall(False)

chip.write_conf()

In [89]:
# SELECT WHICH DOUBLE COLUMNS TO ENABLE
chip['CONF_SR']['EN_PMOS_NOSF'].setall(False)
chip['CONF_SR']['EN_PMOS'].setall(False)
chip['CONF_SR']['EN_COMP'].setall(False)
chip['CONF_SR']['EN_HV'].setall(False)
chip['CONF_SR']['EN_OUT'].setall(False) #ENABLES OR DISABLES THE NORMAL OUTPUT PADS, ACTIVE LOW
chip['CONF_SR']['nEN_OUT'].setall(True) #ENABLES OR DISABLES THE COMPLEMENTARY OUTPUT PADS, ACTIVE LOW
chip['CONF_SR']['EN_HITOR_OUT'].setall(True) #ENABLES OR DISABLES THE NORMAL HITOR PADS, HITOR0-3 =  1-4 flavor, ACTIVE LOW
chip['CONF_SR']['nEN_HITOR_OUT'].setall(True) #ENABLES OR DISABLES THE COMPLEMENTARY HITOR PADS, ACTIVE LOW

#chip['CONF_SR']['EN_PMOS'].setall(True)
chip['CONF_SR']['EN_PMOS'][9] = 1
chip['CONF_SR']['EN_PMOS'][10] = 1
chip['CONF_SR']['EN_PMOS'][11] = 1
chip['CONF_SR']['EN_HITOR_OUT'][1] = 0

# SELECT WHICH PHYSICAL COLUMNS, ROWS, DIAGONALS TO MASK
# TO MASK ONE PIXEL, MASKV, MASKH and MASKD OF THIS PIXEL SHOULD BE 0 (FALSE)
# THE MASKD NUMBER OF THE PIXEL WE WANT TO MASK (or UNMASK), IS GIVEN BY: MASKD = PHYSCOL- PHYSROW
# IF PHYSCOL-PHYSROW<0, then MASKD = 448+PHYSCOL-PHYSROW
#chip['CONF_SR']['MASKD'].setall(False)
#chip['CONF_SR']['MASKH'].setall(False)
#chip['CONF_SR']['MASKV'].setall(False)

# TO USE THE MASK FUNCTION YOU MUST INPUT THE FLAVOR, COLUMN AND ROW
# THE FLAVOR NUMERS IS: 0 FOR PMOS_NOSF, 1 FOR PMOS, 2 FOR COMP, 3 FOR HV

for d in range(28,34):
   chip['CONF_SR']['MASKD'][d] = True
for i in range(99,105):    
   chip['CONF_SR']['MASKH'][i] = True
chip['CONF_SR']['MASKH'][102] = False

# chip['CONF_SR']['MASKD'].setall(True)
# chip['CONF_SR']['MASKH'].setall(True)
# chip['CONF_SR']['MASKV'][224:112] = True
# chip['CONF_SR']['EN_PMOS'].setall(True)

chip.mask(1, 33, 72)
chip.mask(1, 17, 30)
chip.mask(1, 41, 66)
chip.mask(1, 97, 94)
chip.mask(1, 34, 151)
chip.mask(1, 40, 123)
chip.mask(1, 82, 193)

# SELECT WHICH PHYSICAL COLUMS TO INJECT
# INJ_IN_MON_L AND INJ_IN_MON_L SELECT THE LEFT AND RIGHT SPECIAL ANALOG MONITORING PIXELS
chip['CONF_SR']['COL_PULSE_SEL'].setall(False)

chip['CONF_SR']['INJ_IN_MON_L'] = 1 # ENABLE INJECTION FOR THE ANALOG MONITORING PIXELS LEFT SIDE
chip['CONF_SR']['INJ_IN_MON_R'] = 1 # ENABLE INJECTION FOR THE ANALOG MONITORING PIXELS RIGHT SIDE

# SELECT WHICH PHYSICAL ROWS TO INJECT
# THE SPEXIAL PIXELS OUTA_MON3 to OUTA_MON0 CORRESPONT TO ROWS 223 to 220 FOR INJECTION
chip['CONF_SR']['INJ_ROW'].setall(False)
chip['CONF_SR']['INJ_ROW'][223] = True # FOR THE ANALOG MONITORING TOP PIXEL
chip['CONF_SR']['INJ_ROW'][99] = True

# SELECT PHYSICAL COLUMNS AND ROWS FOR INJECTION WITH FUNCTION
chip.enable_injection(1,18,99)

# SELECT PHYSICAL COLUMN(S) FOR HITOR OUTPUT
# nMASKH (SO SETTING MASKH TO FALSE) ENABLES HITOR FOR THE SPECIFIC ROW
chip['CONF_SR']['DIG_MON_SEL'].setall(False)

chip.enable_column_hitor(1,20)

chip.write_conf()

******************INITIALIZE COMMUNICATION, RECEIVE DATA (CHECK IF DATA ARE SENT WITH EACH EXCECUTION)*******************

************CONFIGURE THE FRONT END CURRENT AND VOLTAGE BIASES, CONFIGURE THE HEIGHT OF THE INJECTION PULSE**************

In [90]:
## SET THE INJECTION PULSE AMPLITUDE
## 128-bit DAC (7-bit binary equivalent)
## SET THE VOLTAGES IN ONE HOT ENCODING, ONLY ONE BIT ACTIVE AT A TIME.
## V = (127/1.8)*#BIT
# The default values are VL=44, VH=79, VH-VL=35
# VDAC LSB=14.17mV, Cinj=230aF, 1.43e-/mV, ~710e-
chip.set_vl_dacunits(49,1)
chip.set_vh_dacunits(79,1)

####### CONFIGURE THE FRONT END ######

# SET VRESET_P, THIS IS THE BASELINE OF THE FRONT END INPUT, ONE HOT ENCODING
chip.set_vreset_dacunits(40,1) #1V

## 128-bit DAC (7-bit binary equivalent)
## SET THE CURRENTS USING THERMOMETER ENCODING, I = #BITS_ACTIVE*140nA*SCALING, SCALING IS DIFFERENT FOR EACH CURRENT
## SCALING: IBIAS=10, IDB=16, ITHR=0.125, ICASN=4, IRESET=0.03125
## ACTIVE BITS SHOULD BE SET STARTING FROM THE MIDDLE e.g. for 15 active bits, (128-15)/2=56,5 so 56zeros,15ones,57zeros
## Thus, Ix[71:57] = True

# SET ICASN, THIS CURRENT CONTROLS THE OUTPUT BASELINE, BE CAREFUL NOT TO SET IT TO HIGH
# ALWAYS MONITOR THE POWER AFTER SETTING ICASN. IF VDDD IS SEVERAL mA, REDUCE IT UNTIL IT RETURNS TO 0
# ICASN MAINLY CONTROLS THE THRESHOLD
chip.set_icasn_dacunits(0,1) #4.375nA # approx 1.084V at -3V backbias, 600mV at 0V backbias

# SET IRESET, THIS CURRENT CONTROLS THE RESET RATE OF THE FRONT END INPUT (ALSO THE THRESHOLD)
chip.set_ireset_dacunits(3,1,1) #270pA, HIGH LEAKAGE MODE, NORMAL SCALING, 0 = LOW LEAKAGE MODE, SCALING*0.01

# SET ITHR, THIS CURRENT CONTROLS THE RESET RATE OF THE OUTPUT (AND THE THRESHOLD)
chip.set_ithr_dacunits(15,1) #680pA

# SET ITHR, THIS CURRENT CONTROLS THE BIASING OF THE chip.get_power_status()DISCRI17.050179545339105,MINATOR (AND THE THRESHOLD)
chip.set_idb_dacunits(20,1) #500nA

# SET IBIAS, THIS CURRENT IS THE DC CURRENT OF THE MAIN BRANCH OF THE FRONT END THAT PROVIDES AMPLIFICATION
# IT CONTROLS MAINLY THE RISE TIME
chip.set_ibias_dacunits(100,1) #500nA  # default 50
chip.get_power_status()
############ ENABLE THE DAC CURRENT MONITORING ###########
# chip['CONF_SR']['SWCNTL_DACMONI'] = 0

########## SET THE BIAS CURRENTS OF THE TWO STAGE SOURCE FOLLOWER THAT BUFFERS THE ANALOG MONITORING VOLTAGES #########
# CONTROLS THE RESPONSE TIME AND THE LEVEL SHIFT OF THE BUFFER
# chip['CONF_SR']['SET_IBUFN_L'] = 0b1001
# chip['CONF_SR']['SET_IBUFP_L'] = 0b0101

chip.write_conf()

vl = 1.07948818898V
vh = 1.50468503937V
vreset = 1.12192913386V
icasn = 4.375nA
ireset = 0.13671875nA, high leakage mode
ithr = 2.1875nA
idb = 367.5nA
ibias = 1104.6875nA


In [26]:
chip.get_power_status()

{'VDDA [V]': 1.7825418966900703,
 'VDDA [mA]': 81.9791897275874,
 'VDDA_DAC [V]': 1.7981297475149105,
 'VDDA_DAC [mA]': 3.513856326790666,
 'VDDD [V]': 1.799880103792415,
 'VDDD [mA]': 0.501475573372531,
 'VDDP [V]': 1.788289688,
 'VDDP [mA]': 173.8162655353089}

In [94]:

reload( tjmonopix.scans.simple_scan)

scan =  tjmonopix.scans.simple_scan.SimpleScan(dut=chip)
scan.start(with_timestamp=False,scan_time=10, with_tdc=True, with_tlu=False)
scan.analyze()

2018-03-09 17:14:57,649 - [root           ] - INFO    Initializing SimpleScan
2018-03-09 17:14:57,653 - [root           ] - INFO    Creating ZMQ context
2018-03-09 17:14:57,655 - [root           ] - INFO    Creating socket connection to server tcp://127.0.0.1:5500
2018-03-09 17:14:57,657 - [root           ] - INFO    ScanBase.start:data_send.data_send_init connected=<zmq.sugar.socket.Socket object at 0x7f2c2e173c80>
2018-03-09 17:14:57,660 - [root           ] - INFO    Resetting SRAM FIFO: size = 0
2018-03-09 17:14:57,884 - [root           ] - INFO    Starting FIFO readout
2018-03-09 17:14:57,888 - [root           ] - WARNING SRAM FIFO not empty when starting FIFO readout: size = 704
2018-03-09 17:14:57,891 - [root           ] - INFO    *****SimpleScan is running **** don't forget to start tlu ****
2018-03-09 17:14:57,894 - [root           ] - INFO    time=0s dat=176 rate=67.588k/s
2018-03-09 17:15:07,969 - [root           ] - INFO    Stopped FIFO readout
2018-03-09 17:15:07,971 - [roo

In [81]:
#chip['data_rx'].reset()
chip['data_rx'].set_en(True)

x = chip['fifo'].get_data()

ix = chip.interprete_data(x)

print len(ix)
print(ix)

38
[(19, 101,  6, 19, 0) (23, 101, 43, 53, 0) (18, 101, 21,  3, 0)
 (18, 101, 61, 27, 0) (18, 100, 62, 14, 0) (18, 101, 41, 57, 0)
 (20, 102, 42, 62, 0) (20, 101, 42, 53, 0) (23,  99, 40,  3, 0)
 (22, 104, 32,  0, 0) (22, 103, 32, 19, 0) (18, 104,  7, 25, 0)
 (18, 103,  7, 39, 1) (18,  98, 41, 35, 0) (18,  97, 41, 37, 0)
 (19, 104,  1, 24, 0) (20, 100, 12, 12, 0) (18, 104, 20, 26, 0)
 (18, 100, 26,  5, 0) (22, 106, 25, 49, 0) (23, 105, 26, 32, 0)
 (22, 103, 41,  3, 0) (19, 104, 53, 57, 0) (21, 100, 15, 26, 0)
 (20, 100, 14, 10, 1) (23, 103, 15, 36, 0) (23,  99, 15, 21, 0)
 (18, 103, 52, 57, 0) (19, 103, 51,  5, 0) (18, 102, 62,  8, 0)
 (19,  98, 25, 38, 0) (18,  98, 25, 63, 1) (18,  97, 25, 42, 1)
 (19, 103, 20, 53, 0) (19, 101, 62, 13, 0) (23, 105, 25, 34, 0)
 (21, 102,  0,  7, 0) (22, 102, 63, 30, 1)]


****************************SET THE INJECTION PULSE PARAMETERS, INJECT x times and get the data**************************